In [ ]:
import numpy as np

# Define the grid size
grid_size = 4

# Define the rewards for each state
rewards = np.full((grid_size, grid_size), -1.0)  # Initialize with -1 for all moves
rewards[grid_size - 1, grid_size - 1] = 0  # Goal state reward

# Define value function (initialize to zero)
values = np.zeros((grid_size, grid_size))

# Define parameters
gamma = 0.9  # Discount factor
theta = 1e-4  # Convergence threshold
max_policy_eval_iterations = 10  # Max iterations for policy evaluation
actions = ['up', 'down', 'left', 'right']
action_deltas = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}

# Function to check if a state is terminal
def is_terminal_state(x, y):
    return (x == grid_size - 1 and y == grid_size - 1)

# Function to get the next state given an action
def get_next_state(x, y, action):
    if action in action_deltas:
        delta = action_deltas[action]
        new_x, new_y = x + delta[0], y + delta[1]
        if 0 <= new_x < grid_size and 0 <= new_y < grid_size:
            return new_x, new_y
    return x, y  # No change if out of bounds or invalid action

# Initialize a random policy (for simplicity, let's start with 'right' for all non-terminal states)
policy = np.full((grid_size, grid_size), 'right', dtype=str)
policy[grid_size - 1, grid_size - 1] = 'Goal'  # Terminal state policy

# Policy Iteration Algorithm with truncated policy evaluation
is_policy_stable = False
iteration = 0

while not is_policy_stable:
    # Policy Evaluation
    for eval_iteration in range(max_policy_eval_iterations):
        delta = 0
        new_values = np.copy(values)
        for x in range(grid_size):
            for y in range(grid_size):
                if is_terminal_state(x, y):
                    continue
                action = policy[x, y]
                new_x, new_y = get_next_state(x, y, action)
                reward = rewards[new_x, new_y]
                new_values[x, y] = reward + gamma * values[new_x, new_y]
                delta = max(delta, abs(new_values[x, y] - values[x, y]))
        values = new_values
        if delta < theta:
            break

    # Policy Improvement
    is_policy_stable = True
    for x in range(grid_size):
        for y in range(grid_size):
            if is_terminal_state(x, y):
                continue
            old_action = policy[x, y]
            best_action = None
            best_value = float('-inf')
            for action in actions:
                new_x, new_y = get_next_state(x, y, action)
                reward = rewards[new_x, new_y]
                value = reward + gamma * values[new_x, new_y]
                if value > best_value:
                    best_value = value
                    best_action = action
            policy[x, y] = best_action
            if old_action != best_action:
                is_policy_stable = False

    iteration += 1
    if is_policy_stable:
        break

# Display the final value function
print("Value Function after Convergence:")
print(values)
print(f"Converged in {iteration} iterations.")

# Display the optimal policy
print("\nOptimal Policy:")
for row in policy:
    print(row)
